In [1]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sys.path.insert(0, '../../')
import ccal
%matplotlib inline

==================== Computational Cancer Analysis Library ====================

<10:13:13.705585> Checking dependencies ...
<10:13:13.707704> Using the following packages:
<10:13:13.709408> 	matplotlib (v1.5.1)
<10:13:13.709426> 	numpy (v1.10.4)
<10:13:13.709435> 	pandas (v0.18.0)
<10:13:13.709455> 	rpy2 (v2.7.9)
<10:13:13.709463> 	scikit-learn (v0.17.1)
<10:13:13.709470> 	scipy (v0.17.0)
<10:13:13.709477> 	seaborn (v0.7.0)


In [2]:
nelement = 5
features = ccal.support.make_random_features(10, nelement)
ref = ccal.support.make_random_features(1, nelement)

In [11]:
information_coefficient = ccal.information.information_coefficient
import math
from numpy.random import choice
import scipy.stats as stats
from statsmodels.sandbox.stats.multicomp import multipletests
nsampling = 10

In [5]:
features_nrow, features_ncol = features.shape

nsample = int(0.632 * features_ncol)

sampled_scores = np.empty((features_nrow, nsampling))
for i in range(nsampling):
    sample_indices = choice(features.columns.tolist(), nsample).tolist()
    sample_features = features.ix[:, sample_indices]
    sample_ref = ref.ix[sample_indices]
    for j, (idx, s) in enumerate(sample_features.iterrows()):
        sampled_scores[j, i] = information_coefficient(s, sample_ref)

confidence_intervals = pd.DataFrame(index=features.index, columns=['0.05', '0.5', '0.95'])
z_critical = stats.norm.ppf(q=0.95)
for i, feature in enumerate(sampled_scores):
    mean = feature.mean()
    stdev = feature.std()
    moe = z_critical * (stdev/math.sqrt(feature.size))
    confidence_intervals.iloc[i] = mean - moe, mean, mean + moe

In [7]:
scores = ccal.analyze.compute_against_reference(features, ref)
scores

,Information Coef
Feature 0,-0.402883
Feature 1,0.199355
Feature 2,0.491121
Feature 3,0.314563
Feature 4,-0.580565
Feature 5,-0.351007
Feature 6,-0.521534
Feature 7,0.101534
Feature 8,-0.688293
Feature 9,0.446230


In [10]:
nperm = 5
permutation_scores = np.empty((features_nrow, nperm))
shuffled_ref = np.array(ref)
for i in range(nperm):
    np.random.shuffle(shuffled_ref)
    for j, (idx, s) in enumerate(features.iterrows()):
        permutation_scores[j, i] = information_coefficient(s, shuffled_ref)
permutation_scores

permutation_pval = np.empty(features_nrow)
for i, feature in enumerate(permutation_scores):
    pval = sum(feature > float(scores.iloc[i])) / nperm
    if pval:
        permutation_pval[i] = sum(feature > float(scores.iloc[i])) / nperm
    else:
        permutation_pval[i] = 1/nperm
permutation_pval

../../ccal/information.py:84: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  v = v[overlap] + jitters[i]


array([ 0.8,  0.2,  0.2,  0.4,  1. ,  1. ,  0.8,  0.4,  1. ,  0.2])

In [12]:
multipletests(permutation_pval, method='fdr_bh')

(array([False, False, False, False, False, False, False, False, False, False], dtype=bool),
 array([ 1.        ,  0.66666667,  0.66666667,  0.8       ,  1.        ,
         1.        ,  1.        ,  0.8       ,  1.        ,  0.66666667]),
 0.0051161968918237433,
 0.005)